In [1]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

import sys
sys.path.insert(0, '../impl/utils/')
import voxel_processing as vp

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda')

In [4]:
class PRS_Encoder(nn.Module):
    def __init__(self, ) -> None:
        super(PRS_Encoder, self).__init__()
        
        leaky_ReLU_slope = 0.2
        
        # 32^3x1
        self.conv_layer1 = nn.Conv3d(in_channels=1, out_channels=4, kernel_size=3, padding=1)
        self.max_pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.leaky_relu1 = nn.LeakyReLU(negative_slope=leaky_ReLU_slope)
        
        # 16^3x4
        self.conv_layer2 = nn.Conv3d(in_channels=4, out_channels=8, kernel_size=3, padding=1)
        self.max_pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.leaky_relu2 = nn.LeakyReLU(negative_slope=leaky_ReLU_slope)
        
        # 8^3x8
        self.conv_layer3 = nn.Conv3d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.max_pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.leaky_relu3 = nn.LeakyReLU(negative_slope=leaky_ReLU_slope)
        
        # 4^3x16
        self.conv_layer4 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.max_pool4 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.leaky_relu4 = nn.LeakyReLU(negative_slope=leaky_ReLU_slope)
        
        # 2^3x32
        self.conv_layer5 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.max_pool5 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.leaky_relu5 = nn.LeakyReLU(negative_slope=leaky_ReLU_slope)
        # 1^3x64
    
    def forward(self, voxels):
        out = self.conv_layer1(voxels)
        out = self.max_pool1(out)
        out = self.leaky_relu1(out)
        
        out = self.conv_layer2(out)
        out = self.max_pool2(out)
        out = self.leaky_relu2(out)
        
        out = self.conv_layer3(out)
        out = self.max_pool3(out)
        out = self.leaky_relu3(out)
        
        out = self.conv_layer4(out)
        out = self.max_pool4(out)
        out = self.leaky_relu4(out)
        
        out = self.conv_layer5(out)
        out = self.max_pool5(out)
        out = self.leaky_relu5(out)
        
        return out
        

In [5]:
class PRS_Plane_Predictor(nn.Module):
    def __init__(self, ) -> None:
        super(PRS_Plane_Predictor, self).__init__()
        
        # implicit symmetry planes: 4 features, aX + bY + cZ + d = 0
        self.fc1 = nn.Linear(64, 32)
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(16, 4)
        
    def forward(self, features):
        out = self.fc1(features)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        
        return out

In [6]:
class PRS_Quaterion_Predictor(nn.Module):
    def __init__(self, ) -> None:
        super(PRS_Quaterion_Predictor, self).__init__()
        
        # quaterion rotation: 4 features, a + bi + cj + dk
        self.fc1 = nn.Linear(64, 32)
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(16, 4)
        
    def forward(self, features):
        out = self.fc1(features)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        
        return out

In [7]:
class PRS_SymmDist_Loss(nn.Module):
    '''
    PRS Symmetry Distance Loss
    '''
    def __init__(self, ) -> None:
        super(PRS_SymmDist_Loss, self).__init__()
    
    def forward(self, voxels, grid_points):
        # TODO
        # We 
        pass

In [8]:
class PRS_Reg_Loss(nn.Module):
    '''
    PRS Regularization Loss
    '''
    def __init__(self, ) -> None:
        super(PRS_Reg_Loss, self).__init__()
    
    def forward(self, voxels, grid_points):
        # TODO
        pass

In [9]:
class PRS_Loss(nn.Module):
    def __init__(self, ) -> None:
        super(PRS_Loss, self).__init__()
        self.symmetry_loss = PRS_SymmDist_Loss()
        self.reg_loss = PRS_Reg_Loss()
        self.reg_loss_weight = Variable([1], requires_grad=True)
    
    def forward(self, voxels, distances):
        # TODO
        pass

In [10]:
class PRSNet(nn.Module):
    def __init__(self, ) -> None:
        super(PRSNet, self).__init__()
        self.encoder = PRS_Encoder()
        self.plane_predictor = PRS_Plane_Predictor()
        self.quaterion_predictor = PRS_Quaterion_Predictor()
        
    def forward(self, voxels, distances):
        # TODO
        pass

In [3]:
# Create datasets
vp.preprocess_files('../data/obj_test_data/', '../data/voxel_data')

Processing: ../data/obj_test_data/voxel_grid_1.obj -> ../data/voxel_data/voxel_grid_0
Processing: ../data/obj_test_data/voxel_grid_0.obj -> ../data/voxel_data/voxel_grid_1
2 file(s) have been processed.


In [6]:
# Load datasets
data = vp.read_data_from_path('../data/voxel_data/')
data

Reading: ../data/voxel_data/voxel_grid_1.obj
Reading: ../data/voxel_data/voxel_grid_0.omap
Reading: ../data/voxel_data/voxel_grid_1.offsetvec
Reading: ../data/voxel_data/voxel_grid_0.gridpoints
Reading: ../data/voxel_data/voxel_grid_0.obj
Reading: ../data/voxel_data/voxel_grid_1.omap
Reading: ../data/voxel_data/voxel_grid_0.offsetvec
Reading: ../data/voxel_data/voxel_grid_1.gridpoints
2 dataset(s) have been processed. 


array([-0.01545852,  0.8088349 ,  0.15418914], dtype=float32)